<a href="https://colab.research.google.com/github/BhaskarFysik/AlgoShared/blob/main/bh_rsi_oscillator_and_smacross.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 7.7 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=8a81fc6065c301517bbfba327f1d6ab98f9b71e6d688be20c98ef1963d869fb0
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 8.2 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173833 sha256=ae376e77f8596dd117889cd98682513442b3f2b5397426cb0f05e7e7c63a8c2c
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
#from backtesting.test import GOOG

symbol='SBIN'
df = yf.Ticker(f"{symbol}.NS").history(period="5y", interval="1d")
#df=pd.DataFrame(GOOG)
def RSI(values,n):
     return ta.rsi(close=df["Close"], length=n)
def SMA(values,n):
     return pd.Series(values).rolling(n).mean()



class RsiOscillator(Strategy):
    upper_bound=70
    lower_bound=30
    n1=20
    n2=50
    rsi_window=14
    def init(self):
        self.rsi=self.I(RSI,self.data.Close,self.rsi_window)
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self):

        if crossover(self.rsi,self.upper_bound) and self.sma2>self.sma1:
            self.position.close()
        elif crossover(self.lower_bound,self.rsi) and self.sma2<self.sma1:
            self.buy()

bt=Backtest(df,RsiOscillator, cash=10000)
stats=bt.optimize(
    upper_bound=range(10,85,5),
    lower_bound=range(10,85,5),
    n1=range(5, 30, 5),
    n2=range(10, 70, 5),
    #rsi_window=range(10,30,2),
    maximize='Equity Final [$]',
    constraint=lambda param: param.upper_bound>param.lower_bound and param.n1 < param.n2)
#stats=bt.run()
print(stats)
bt.plot()

/usr/local/lib/python3.7/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 5250 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/18 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.7/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Start                     2017-11-27 00:00...
End                       2022-11-25 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                   52.874494
Equity Final [$]                 44053.090921
Equity Peak [$]                  44413.090921
Return [%]                         340.530909
Buy & Hold Return [%]               85.483269
Return (Ann.) [%]                   35.332614
Volatility (Ann.) [%]                32.78554
Sharpe Ratio                         1.077689
Sortino Ratio                        2.425628
Calmar Ratio                         1.436458
Max. Drawdown [%]                  -24.597033
Avg. Drawdown [%]                   -4.715884
Max. Drawdown Duration      425 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                     163.866373
Worst Trade [%]                      -1.89173
Avg. Trade [%]                    

Row(id='1500', ...)

In [ ]:
print(stats["_strategy"].upper_bound)
print(stats["_strategy"].lower_bound)
print(stats["_strategy"].n1)
print(stats["_strategy"].n2)
print(stats["_strategy"].rsi_window)

60
45
20
55
26
